In [1]:
import mysql.connector
import pandas as pd
import re
import demoji #To process emojis
demoji.download_codes()
from mysql.connector import errorcode

try:
    cnx = mysql.connector.connect(user='root', password='cwzwbx1995',
                              host='127.0.0.1',
                              database='data')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)

... OK (Got response in 0.19 seconds)
Writing emoji data to /Users/tinabaixinn/.demoji/codes.json ...
... OK


In [2]:
cursor = cnx.cursor()

In [3]:
# 1. The PostId of the post with the most engagements is:
    
query = ("SELECT PostId FROM performance_data "
         "WHERE Engagements = (SELECT MAX(Engagements) FROM performance_data)")
cursor.execute(query)
for row in cursor:
    for field in row:
         print("The PostId of the post with the most engagements is {}".format(field))

The PostId of the post with the most engagements is 28144


In [4]:
# 2. What is the average engagement rate of multi-image posts? :

query = ("SELECT AVG(EngagementRate) FROM performance_data "
         "INNER JOIN post_data ON performance_data.PostId = post_data.PostId "
         "WHERE post_data.Type = 'Multi Image'")
cursor.execute(query)
for row in cursor:
    for field in row:
         print("The average engagement rate of multi image posts is {:.2f}%".format(field * 100))

The average engagement rate of multi image posts is 1.10%


In [5]:
# 3. What post type has the highest average reach per post? 

query = ("SELECT Type, AVG(Reach) FROM performance_data "
         "INNER JOIN post_data ON performance_data.PostId = post_data.PostId "
         "GROUP BY Type "
         "ORDER BY AVG(Reach) DESC "
         "LIMIT 1")
cursor.execute(query)
for row in cursor:
     print("{} post type has the highest average reach per post".format(row[0]))

Multi Image post type has the highest average reach per post


In [6]:
# 4. On what date were the most posts published?

query = ("SELECT CAST(PublishTime AS DATE), COUNT(post_data.PostId) From post_data "
         "GROUP BY CAST(PublishTime AS DATE) "
         "ORDER BY COUNT(post_data.PostId) DESC "
         "LIMIT 1")
cursor.execute(query)
for row in cursor:
     print("The date on which the most posts were published is {}".format(row[0]))

The date on which the most posts were published is 2020-01-11


In [7]:
# 5. During which hour of the day is the average number of impressions per post greatest?

from datetime import datetime 
from datetime import timedelta 
query = ("SELECT date_format(PublishTime,'%h%p') AS hour_interval, "
         "AVG(performance_data.Impressions) "
         "FROM post_data "
         "INNER JOIN performance_data ON post_data.PostId = performance_data.PostId "
         "GROUP BY hour_interval "
         "ORDER BY AVG(performance_data.Impressions) DESC "
         "LIMIT 1")
cursor.execute(query)
for row in cursor:
     print("The greatest average number of impressions per post is during {}".format(row[0]))

The greatest average number of impressions per post is during 06AM


In [8]:
# 6. What is the UserId of the user who created the most posts? 

query = ("SELECT UserId, COUNT(PostId) "
         "FROM post_data "
         "GROUP BY UserId "
         "ORDER BY COUNT(PostId) DESC " 
         "LIMIT 1")
cursor.execute(query)
for row in cursor:
    print(" User who created the most posts is " + row[0] + " ")

 User who created the most posts is dae201f8-6fed-4054-8f4b-8a1cee435e99


In [3]:
# 7. What word appears most frequently in the post messages of posts made on 1/3/20? 
# Ignore capitalization, punctuation, and emojis.

query = ("SELECT Message, PublishTime FROM post_data "
         "WHERE PublishTime < '2020-01-04' AND PublishTime > '2020-01-02'")
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
df.columns = ['Message', 'PublishTime']

In [4]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,wordfreq)))

In [5]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [7]:
wordstring = ""
for index, row in df.iterrows():
    s = re.sub(r'[^\w\s]', '', row['Message'].lower()) #regex of punctuation, to remove punctuations
    wordstring += demoji.replace(s, '')
wordlist = wordstring.split()
freqdict = wordListToFreqDict(wordlist)
sortedFreqDict = sortFreqDict(freqdict)

In [8]:
print("\"{}\" appears most frequently in the post messages of posts made on 1/3/20.".format(sortedFreqDict[0][1]))

"to" appears most frequently in the post messages of posts made on 1/3/20.


In [14]:
# 8.What is the median character count of the post messages? 
# Count each emoji as one character and count spaces as one character.
query = ("SELECT Message FROM post_data")
cursor.execute(query)
rows = cursor.fetchall()
df8 = pd.DataFrame(rows)
df8.columns = ['Message']

In [15]:
character_count = []
for index, row in df8.iterrows():
    s = demoji.replace(row['Message'], ' ')
    character_count.append(len(s))
df8['Character Count'] = character_count
df8

,Message,Character Count
0,"These are games, what’s not a game is you work...",247
1,Please have a safe New Year from the Villa Par...,117
2,🎉Only a few more hours to get your blackeye pe...,86
3,"PARTY ON WAYNE, PARTY ON GARP!At your local Co...",63
4,Bring in the New Year with a bang at your Hend...,74
...,...,...
46849,Come on out to your local Auto Care Center at ...,104
46850,Looking for quick snacks for Sunday’s game? Yo...,194
46851,Need help to arrange party this weekend? Let o...,183
46852,"If you can't quote one sitcom completely, do y...",144


In [16]:
print("The median character count of the post messages is {}"
      .format(df8['Character Count'].median()))

The median character count of the post messages is 152.0


In [17]:
# 9. 95% of posts have a message character count less than 
# (count each emoji as one character and count spaces as one character)
df9 = df8.sort_values(by='Character Count')

In [18]:
row_index = int(df9.shape[0] * 0.05)
print("95% of posts have a message character count less than {}"
     .format(df9.iloc[row_index]['Character Count']))

95% of posts have a message character count less than 56


In [19]:
# 10. What percentage of video posts have a video completion rate equal to 100% 
# (VideoCompletionRate = VideoCompletions / VideoViews)?
query = ("SELECT PostId, VideoCompletions, VideoViews FROM performance_data")
cursor.execute(query)
rows = cursor.fetchall()
df10 = pd.DataFrame(rows)
df10.columns = ['PostId', 'VideoCompletions', 'VideoViews']
df10

,PostId,VideoCompletions,VideoViews
0,1,0,0
1,2,0,0
2,3,0,0
3,4,0,0
4,5,0,0
...,...,...,...
46849,47597,0,0
46850,47598,0,0
46851,47599,0,0
46852,47600,0,0


In [20]:
row_number_completion = df10[df10['VideoCompletions'] == df10['VideoViews']].loc[df10['VideoViews'] > 0].shape[0]
percentage = row_number_completion/df10.shape[0] * 100
print("{:.2f}% of video posts have a video completion rate equal to 100%".format(percentage))

0.04% of video posts have a video completion rate equal to 100%
